<a href="https://colab.research.google.com/github/TM-M150/Book-Genre-Classification/blob/master/Best_TFIDF-Vectorizer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Data/book32-listing.csv',encoding = "ISO-8859-1")

In [7]:
###dsa as as 
columns = ['Id', 'Image', 'Image_link', 'Title', 'Author', 'Class', 'Genre']
data.columns = columns

In [8]:
books = pd.DataFrame(data['Title'])
author = pd.DataFrame(data['Author'])
genre = pd.DataFrame(data['Genre'])

In [9]:
data['Author'] = data['Author'].fillna('No Book')
data['Title'] = data['Title'].fillna('No Book')

In [10]:
print (len(books))
print (len(genre))
genre.head(2)
books[50000:50110]

207571
207571


,Title
50000,"The Ultimate Sock Puppet Book: Clever Tips, Tr..."
50001,"Puppets, Masks, and Performing Objects"
50002,Felt Board Fingerplays with Patterns & Activit...
50003,Jim Henson and Philosophy: Imagination and the...
50004,Wael Shawky: Cabaret Crusades
...,...
50105,"4G: LTE/LTE-Advanced for Mobile Broadband, Sec..."
50106,Spotlight Synthetic Aperture Radar: Signal Pro...
50107,"Heterogeneous Cellular Networks: Theory, Simul..."
50108,Plain-English Study Guide for the General Radi...


In [11]:
print (len(books))
print (len(genre))
#print (len(all_books))
#print (len(all_genres))

207571
207571


In [12]:
genre['Genre'].unique()

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

In [13]:
from sklearn.preprocessing import LabelEncoder

feat = ['Genre']
for x in feat:
    le = LabelEncoder()
    le.fit(list(genre[x].values))
    genre[x] = le.transform(list(genre[x]))
    

In [14]:
genre['Genre'].unique()

array([ 3,  6, 30, 19, 26, 24, 15, 17, 12, 11,  8,  9,  0, 10, 20, 27,  7,
       18, 25, 13,  2, 16,  1, 14, 21, 22,  5, 23, 28, 29,  4, 31])

In [15]:
le.inverse_transform([0])[0]

'Arts & Photography'

In [16]:
data['everything'] = pd.DataFrame(data['Title'] + ' ' + data['Author'])
print (data['everything'].head(5))
#for i,j in enumerate(data['everything']):
#    data['everything'][i] = data['everything'][i].lower()
#data['everything'] = data['everything'].str.lower()

0         Doug the Pug 2016 Wall Calendar Doug the Pug
1    Moleskine 2016 Weekly Notebook, 12M, Large, Bl...
2    365 Cats Color Page-A-Day Calendar 2016 Workma...
3     Sierra Club Engagement Calendar 2016 Sierra Club
4     Sierra Club Wilderness Calendar 2016 Sierra Club
Name: everything, dtype: object


In [17]:
def change(t):
    t = t.split()
    return ' '.join([(i) for (i) in t if i not in stop])

In [21]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
stop[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
#remove_list = ['the','at','or']
#for i,j in enumerate(data['everything']):
#    t = data['everything'][i].split()
#    data['everything'][i] = ' '.join([str(i) for (i) in t if i not in stop])

In [22]:
data['everything'].apply(change)

0                      Doug Pug 2016 Wall Calendar Doug Pug
1         Moleskine 2016 Weekly Notebook, 12M, Large, Bl...
2         365 Cats Color Page-A-Day Calendar 2016 Workma...
3          Sierra Club Engagement Calendar 2016 Sierra Club
4          Sierra Club Wilderness Calendar 2016 Sierra Club
                                ...                        
207566    ADC Map People Washington D.C.: Street Map Boo...
207567    Washington, D.C., Then Now: 69 Sites Photograp...
207568    The Unofficial Guide Washington, D.C. (Unoffic...
207569    Washington, D.C. For Dummies (Dummies Travel) ...
207570    Fodor's Where Weekend Around Boston, 1st Editi...
Name: everything, Length: 207571, dtype: object

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
vectorizer = TfidfVectorizer(min_df=2, max_features=70000, strip_accents='unicode',lowercase =True,
                            analyzer='word', token_pattern=r'\w+', use_idf=True, 
                            smooth_idf=True, sublinear_tf=True, stop_words = 'english')
vectors = vectorizer.fit_transform(data['everything'])
vectors.shape

(207571, 58432)

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(vectors, genre['Genre'], test_size=0.02)

In [28]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(203419, 58432)
(203419,)
(4152, 58432)
(4152,)


In [29]:
type(books)

pandas.core.frame.DataFrame

## Gaussian NB

In [30]:
clf = MultinomialNB(alpha=.45)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.5059217004058675
0.6088631984585742


## Logistic Regression

In [31]:
from sklearn import linear_model
clf = linear_model.LogisticRegression(solver= 'sag',max_iter=200,random_state=450)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

0.6019321492280649
0.6486030828516378


In [32]:
text = ['I too had a Love Story']
text[0] = text[0].lower()
#text = list(text)
s = (vectorizer.transform(text))
#s = vectorizer.fit_transform(df)
print (s.shape)
d = (clf.predict(s))

(1, 58432)


In [33]:
le.inverse_transform(d)[0]

'Biographies & Memoirs'

## Saving Model

In [36]:
import joblib
joblib.dump(clf, '/content/best.pkl')
print("Model saved")

Model saved


In [37]:
clf = joblib.load('/content/best.pkl')

In [38]:
clf

LogisticRegression(max_iter=200, random_state=450, solver='sag')

## Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation='logistic', alpha=0.00003, batch_size='auto',
                   beta_1=0.9, beta_2=0.999, early_stopping=False,
                   epsilon=1e-08, hidden_layer_sizes=(20,), learning_rate='constant',
                   learning_rate_init=0.003, max_iter=200, momentum=0.9,
                   nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
                   solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
                   warm_start=False)
clf.fit(X_train, y_train) 
pred = clf.predict(X_test)
print (metrics.f1_score(y_test, pred, average='macro'))
print (metrics.accuracy_score(y_test, pred))

In [ ]:
text = ['Until It Fades: A Novel']
#text = list(text)
s = (vectorizer.transform(text))
#s = vectorizer.fit_transform(df)
print (s.shape)
d = (clf.predict(s))

(1, 117281)


In [ ]:
le.inverse_transform(d)[0]

'Literature & Fiction'

## SVM

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(data=X_train, label = y_train)
dtest = xgb.DMatrix(data=X_test)

In [ ]:
params = {
    'objective':'multi:softmax',
    'eval_metric':'mlogloss',
    'eta':0.025,
    'max_depth':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'min_child_weight':5,
    'num_class': 32
    
}

In [ ]:
bst = xgb.cv(params, dtrain, num_boost_round=100,
early_stopping_rounds=40, nfold=5, verbose_eval=10)

In [ ]:
bst_train = xgb.train(params, dtrain, num_boost_round=10) 

In [ ]:
p_test = bst_train.predict(dtest)

In [ ]:
print (metrics.f1_score(y_test, p_test, average='macro'))
print (metrics.accuracy_score(y_test, p_test)) 

In [ ]:
text = ['Fifty Shades of Grey']
#text = list(text)
s = (vectorizer.transform(text))
#s = vectorizer.fit_transform(df)
print (s.shape)
p = (bst_train.predict(s))

In [ ]:
le.inverse_transform(d)[0]